In [4]:
# keras CNN for text classification
# with sklearn-style API + hyperopt

from kerastext import KerasVectorizer

Using Theano backend.
WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.


In [3]:
# load some data

import pandas as pd
import numpy as np
data = pd.read_csv('RCTDataClean.csv', encoding='latin-1')
ab = [r if isinstance(r, str) else "" for r in list(data.AB)]
ti = [r if isinstance(r, str) else "" for r in list(data.TI)]

X_text = [r[0] + "\n\n" + r[1] for r in zip(ab, ti)]
y = np.array(data.Relevant)    
    

/Users/iain/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# get some embeddings from somewhere to use to initialise the CNN embedings
from gensim.models import Word2Vec
word_embeddings = Word2Vec.load_word2vec_format('PubMed-w2v.bin', binary=True)

In [6]:
# the vectorizer makes a subset of embeddings indexed correctly
# alongside the text, pass the gensim to it when instantiating
vec = KerasVectorizer(embedding_inits=word_embeddings)

In [7]:
X = vec.fit_transform(X_text)

# this generates standard keras-style feature vectors for text, which is:
# n*m matrix with n=number documents, and m=maximum token length across all documents
# documents < maximum length are left-padded with 0's
# 1 = start of sequence 
# 2 = out of vocabulary token (which only applys to later transforms, fit_transform learns the vocab so nothing is OOV)
# 3+ = integer representing token index in order of frequency, for vocab size k, 3 represents most frequent token, 3+k represents least frequent 

In [11]:
# and for example... text and the X...
print(X_text[0])

print(X[1])

Study design: A retrospective review of prospectively collected data in an academic institution. Objective: To evaluate the safety and efficacy of a new type of titanium mesh cage (TMC) in single-level, anterior cervical corpectomy and fusion (ACCF). Methods: Fifty-eight patients consecutive with cervical spondylotic myelopathy (CSM) from cervical degenerative spondylosis and isolated ossification of the posterior longitudinal ligament were treated with a single-level ACCF using either a new type of TMC (28 patients, group A) or the traditional TMC (30 patients, group B). We evaluated the patients for TMC subsidence, cervical lordosis (C2-C7 Cobb and Cobb of fused segments) and fusion status for a minimum of 30 months postoperatively based on spine radiographs. In addition, neurologic outcomes were evaluated using the Japanese Orthopedic Association (JOA) scores. Neck pain was evaluated using a 10-point visual analog scale (VAS). Results: The loss of height of the fused segments was le

In [42]:
# save it for the other notebook...
# (you could run the CNN straight from here without this, just to make it easier
# to run the main model on a GPU machine)
np.savez_compressed('embase_matrices.npz', X=X, y=y, emb=kvec.embedding_weights)